An Introduction to Analytical Modeling with Charm
========================================


In this tutorial, we first introduce the basic usage of the language Charm to express analytical models and then we use what we learned to create actual architectural analytical models and do some design space exploration.

## How to speak Charm

There are three major components in a Charm program: **types**, **model specifications** and **controls**.


Let's first create a type for all **positive integers**:

In [4]:
typedef I+ : int i
    i > 0

There you go! 
From this point on, in our Charm code, we can simply use **I+** to set the type of a variable in our model as positive integers.

Now let's make a relatively more complicated type for a fraction between 0 and 1:

In [5]:
typedef Fraction : float r
    r >= 0
    r <= 1

Now we have another type **Fraction** which represents a real number r (approximated by *float*) $r \in [0, 1]$

You may now wondering why do we bother with these types and how can we use them in a model? Well, let's first describe a very simple **model** that we'd like to express in Charm.

Let's say we are developing a simple host-accelerator type system:

![A simple Host-Accelerator System](figs/intro/host_accelerator.png)

Given this system, let's say we have an application running on this system. For simplicity, we ignore most details and only say that this application can be partitioned into two parts:
* A normal executaion phase
* An acceleration phase

The application finishes once the two phases are completed.
We further assume that the normal execution phase is exected on the host, while the acceleration phase is, of course, offloaded to the accelerator.

Now, the question we ask is *"What is the overall exection speedup when we employ an Accelerator with an acceleration rate $\alpha$ versus a vanilla host-only system?"*

The speedup can be thought of as an overall effectiveness of adding an accelerator in the system.

To quantify this speedup, we normalize the execution time when executed entirely on the host as 1, and we characterize the execution time of the acceleration phase as *f*. Apparently, $f \in [0, 1]$. In extreme cases, the entire application is offload-able or none of its execution is.

For those who are familiar with Amdahl's Law, the following equation should already have popped up:

$ Speedup = \frac{1}{1 - f + \frac{f}{\alpha}} $

And we'd very much like to put the above equation in Charm!

Now, let's put down our **model specification** for this host-accelerator system.

Like everything in this world, we first have to name it:

In [8]:
define host_accelerator_model:

Now, we need to spell out what are the quantities we use in this model:

In [9]:
    acceleration_phase : Fraction
    acceleration_rate : I+
    speedup : R+

Each line corresponds to one quantity of a certain **type**. Guess what, we already defined **Fraction** upfront! It represents a real number between 0 and 1, what a coincidence!

Here we make another assumption that the acceleration rate is a positive integer. Think of it as the number of processing units (cores if you'd like) in the accelerator and we can always make our acceleration_phase utilize that much parallelism. Speaking of coincidence, we can now use **I+** to accurately type acceleration_rate! Who would have seen that coming!

For the final metric, speedup, we'd like a real number instead of an interger. What's more, it better be a positive number, otherwise buttons pushed and brains exploded. I'll leave the definition for **R+** to you.

And finally, surprise surprise, we write down the equation:

In [11]:
speedup = 1 / (1 - acceleration_phase + acceleration_phase / acceleration_rate)

Wait a second, this is not quite exactly concise and pretty to look at, what we'd ideally want is something very similar to the simple equation we had!

Well, I'm glad you asked! In Charm, this can be easily achieved with *local aliases*:


In [23]:
    acceleration_phase : Fraction as f
    acceleration_rate : I+ as a
    speedup : R+

If we define our variables with an *as*, we can write our equation as:

In [24]:
speedup = 1 / (1 - f + f / a)

How neat is this!

Now we have a **model specification**, along with **types**, it looks like the following:


In [14]:
typedef I+ : int i
    i > 0
    
typedef R+ : float r
    r > 0

typedef Fraction : float r
    r >= 0
    r <= 1

define host_accelerator_model:
    acceleration_phase : Fraction as f
    acceleration_rate : I+ as a
    speedup : R+
    
    speedup = 1 / (1 - f + f / a)

We still haven't answered our qeustion yet though, given an accelerator with 16 cores, along with an application half of which can be parallelized, what exactly is speedup?

That's what **control** is for! Let's first tell Charm what model we are using:

In [15]:
given host_accelerator_model

Couldn't be simpler, now let's tell Charm what we assume:

In [16]:
assume acceleration_phase = 0.5
assume acceleration_rate = 16

Note that we use the **exact name** as defined in the model (not the aliases because those are local). The first line says half of the application can be accelerated and the second line says there are 16 cores in the accelerator.

Finally, let's put everything together and tell Charm to get to work:

In [26]:
typedef I+ : int i
    i > 0
    
typedef R+ : float r
    r > 0

typedef Fraction : float r
    r >= 0
    r <= 1

define host_accelerator_model:
    acceleration_phase : Fraction as f
    acceleration_rate : I+ as a
    speedup : R+
    
    speedup = 1 / (1 - f + f / a)

given host_accelerator_model
assume acceleration_phase = 0.5
assume acceleration_rate = 16

explore speedup

{'raw': defaultdict(<class 'list'>, {(0.5, None): [nan], (0.5, 16): [1.8823529411764706]}), 'img': []}

Voila! Charm does what we expect it to: compute the speedup and spit it out in some format, which is not particularly interesting...well, you've seen what a basic Charm program looks like and does, let's move on to something a bit more interesting!

## How to speak Charm in a charming way

Imagine this, the performance of the host and the accelerator in our host-accelerator system depend on how much hardware resource (think of chip area) put into it, how crazy is that!

Now let's say performance P and the chip area of the accelerator A has the following relationship:

$P = \sqrt{A}$

For those who know Pollack's Rule, this may look pretty familiar.

Let's be crazier and say we wanna this and study how the overall system speedup looks like if we have a fixed chip area to allocate between the host and the accelerator in our system.

## How to speak Charm in a practical way

In this part, we are gonna use a simpel analytical model in the paper [Understanding and Estimating Architectural Risk](https://dl.acm.org/citation.cfm?id=3123939.3124541) to demonstrate how to charm up actual analytical architecture models.
